In [1]:
from deap import gp
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../..'))  # or the full path to the "project" directory. This hack should be really fixed
from gpbr.gp.funcs import pow2, sqrtabs, expplusone


pset = gp.PrimitiveSet("main", 1)
pset.addPrimitive(np.add, 2)
# pset.addPrimitive(np.subtract, 2)
pset.addPrimitive(np.multiply, 2)
pset.addPrimitive(np.cos, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(sqrtabs, 1)
# pset.addPrimitive(pow2, 1)
pset.addPrimitive(expplusone, 1)
pset.addEphemeralConstant('rand', (np.random.rand, 1)[0])
# pset.addTerminal(np.pi, 'pi')

pset.renameArguments(ARG0="s")

In [2]:
from deap import creator, base, tools, algorithms
creator.create("FitnessMin", base.Fitness, weights=(-1.0,), subtree_values=None)
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [3]:
toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=3, max_=6)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

In [4]:
def is_2pi_periodic(values, tolerance=1e-5):
    if isinstance(values, (np.integer, int, float)):
        return True
    return np.allclose(values[0], values[-1], atol=tolerance)

In [5]:
interval = np.linspace(0, 2*np.pi, 100, endpoint=True)

In [6]:
ind1 = toolbox.individual()
print(ind1)

sqrtabs(cos(cos(s)))


In [7]:
ind2 = toolbox.individual()
print(ind2)

multiply(sqrtabs(multiply(multiply(cos(0.5363023413412188), cos(0.7097879091999092)), sin(cos(0.08763838998087636)))), sqrtabs(sqrtabs(sin(s))))


In [8]:
from gpbr.gp.evaluators import evaluate_subtrees
ind1.subtree_values = evaluate_subtrees(ind1, pset, s=interval)
ind2.subtree_values = evaluate_subtrees(ind2, pset, s=interval)


In [9]:
def print_periodic_subtrees(individual):
    for i, vals in enumerate(individual.subtree_values):
        if is_2pi_periodic(vals):
            print(gp.PrimitiveTree(individual[individual.searchSubtree(i)]))

In [10]:
print_periodic_subtrees(ind1)

sqrtabs(cos(cos(s)))
cos(cos(s))
cos(s)


In [11]:
print_periodic_subtrees(ind2)

sqrtabs(multiply(multiply(cos(0.5363023413412188), cos(0.7097879091999092)), sin(cos(0.08763838998087636))))
multiply(multiply(cos(0.5363023413412188), cos(0.7097879091999092)), sin(cos(0.08763838998087636)))
multiply(cos(0.5363023413412188), cos(0.7097879091999092))
cos(0.5363023413412188)
0.5363023413412188
cos(0.7097879091999092)
0.7097879091999092
sin(cos(0.08763838998087636))
cos(0.08763838998087636)
0.08763838998087636
sqrtabs(sin(s))
sin(s)


In [12]:
def cx_perioic_subtrees(ind1, ind2):
    """Crossover that exchanges only 2pi-periodic subtrees between individuals."""
    # Find periodic subtrees in both individuals
    periodic_indexes1 = [i for i, vals in enumerate(ind1.subtree_values) if is_2pi_periodic(vals) and i != 0]
    periodic_indexes2 = [i for i, vals in enumerate(ind2.subtree_values) if is_2pi_periodic(vals) and i != 0]
    
    if not periodic_indexes1 or not periodic_indexes2:
        return ind1, ind2  # No periodic subtrees to exchange
    
    # Select random periodic subtree from each individual
    idx1 = np.random.choice(periodic_indexes1)
    idx2 = np.random.choice(periodic_indexes2)
    
    # Get the subtrees
    subtree1 = ind1[ind1.searchSubtree(idx1)]
    subtree2 = ind2[ind2.searchSubtree(idx2)]
    
    # Swap the subtrees
    ind1[ind1.searchSubtree(idx1)] = subtree2
    ind2[ind2.searchSubtree(idx2)] = subtree1
    
    return ind1, ind2

In [13]:
ind1, ind2 = cx_perioic_subtrees(ind1, ind2)

In [14]:
print(ind1)

sqrtabs(sqrtabs(sin(s)))


In [15]:
print(ind2)

multiply(sqrtabs(multiply(multiply(cos(0.5363023413412188), cos(0.7097879091999092)), sin(cos(0.08763838998087636)))), sqrtabs(cos(cos(s))))


In [87]:
[str(gp.PrimitiveTree(ind[ind.searchSubtree(i)])) for i in range(len(ind))]

['cos(cos(expplusone(add(sin(s), sin(0.5960927048090293)))))',
 'cos(expplusone(add(sin(s), sin(0.5960927048090293))))',
 'expplusone(add(sin(s), sin(0.5960927048090293)))',
 'add(sin(s), sin(0.5960927048090293))',
 'sin(s)',
 's',
 'sin(0.5960927048090293)',
 '0.5960927048090293']